# Accessibility Atlas -- Data Exploration

Disability demographics, web accessibility compliance, assistive technology usage, and related datasets.

**Datasets included:**
- US Census disability by county (3,222 counties, ACS 2022)
- US Census disability by state (52 states, ACS 2021)
- WHO Healthy Life Expectancy (185 countries, 2000-2021)
- WebAIM Million 2025 (top 1M website accessibility audit)
- WebAIM Screen Reader Survey #10 (1,539 respondents, 2024)
- ADA digital accessibility lawsuits (2017-2024)
- Section 508 federal compliance (245 agencies)
- VizWiz VQA (4,319 blind user questions)
- WLASL sign language index (2,000 signs)

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 6)
matplotlib.rcParams['font.family'] = 'sans-serif'

## 1. US Disability by County (Census ACS 2022)

3,222 US counties with disability rates broken down by 6 types: hearing, vision, cognitive, ambulatory, self-care, and independent living.

In [ ]:
counties = pd.read_csv("census_disability_by_county_2022.csv", dtype={"fips": str})
print(f"Counties: {len(counties):,}")
print(f"Total US population covered: {counties['total_population'].sum():,}")
print(f"Total with disability: {counties['disability_total'].sum():,}")
print(f"\nDisability rate range: {counties['disability_rate'].min():.1f}% - {counties['disability_rate'].max():.1f}%")
counties.head()

In [ ]:
# Top 15 counties by disability rate
top = counties.nlargest(15, "disability_rate")[["county_name", "total_population", "disability_rate"]]
fig, ax = plt.subplots(figsize=(12, 6))
ax.barh(range(len(top)), top["disability_rate"].values, color="#2563eb")
ax.set_yticks(range(len(top)))
ax.set_yticklabels(top["county_name"].values)
ax.set_xlabel("Disability Rate (%)")
ax.set_title("US Counties with Highest Disability Rates (ACS 2022)")
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Disability type breakdown (national totals)
types = {
    "Hearing": counties["hearing_disability"].sum(),
    "Vision": counties["vision_disability"].sum(),
    "Cognitive": counties["cognitive_disability"].sum(),
    "Ambulatory": counties["ambulatory_disability"].sum(),
    "Self-care": counties["self_care_disability"].sum(),
    "Indep. Living": counties["independent_living_disability"].sum(),
}
fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.bar(types.keys(), [v/1e6 for v in types.values()], color="#2563eb")
ax.set_ylabel("People (millions)")
ax.set_title("US Disability by Type (Census ACS 2022)")
for bar, val in zip(bars, types.values()):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
            f"{val/1e6:.1f}M", ha='center', fontsize=10)
plt.tight_layout()
plt.show()

## 2. WHO Healthy Life Expectancy (2000-2021)

Years of healthy life (HALE) for 185 countries across 22 years. The gap between life expectancy and HALE represents years lost to disability.

In [ ]:
hale = pd.read_csv("who_hale_long.csv")
print(f"Records: {len(hale):,}")
print(f"Countries: {hale['country_code'].nunique()}")
print(f"Years: {hale['year'].min()}-{hale['year'].max()}")
print(f"\nHALE range: {hale['hale_years'].min():.1f} - {hale['hale_years'].max():.1f} years")

region_avg = hale.groupby(["region", "year"])["hale_years"].mean().reset_index()
fig, ax = plt.subplots(figsize=(12, 6))
for region in sorted(region_avg["region"].unique()):
    subset = region_avg[region_avg["region"] == region]
    ax.plot(subset["year"], subset["hale_years"], label=region, linewidth=2)
ax.set_xlabel("Year")
ax.set_ylabel("Healthy Life Expectancy (years)")
ax.set_title("WHO Healthy Life Expectancy by Region (2000-2021)")
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
plt.tight_layout()
plt.show()

## 3. WebAIM Million 2025

Automated WCAG accessibility analysis of the top 1,000,000 website home pages.

In [ ]:
with open("webaim_million_2025.json") as f:
    webaim = json.load(f)

print(f"Pages tested: {webaim['summary']['total_pages_tested']:,}")
print(f"Total errors: {webaim['summary']['total_errors_detected']:,}")
print(f"Avg errors/page: {webaim['summary']['average_errors_per_page']}")
print(f"Pages with failures: {webaim['summary']['pages_with_wcag_failures_pct']}%")
print("\nMost common WCAG failures:")
for f_item in webaim["common_failures"]:
    print(f"  {f_item['type']:30s} {f_item['pages_affected_pct']}% of pages")

In [ ]:
trends = webaim["yearly_trends"]
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(trends["years"], trends["pages_with_failures_pct"], 'o-', color="#dc2626", linewidth=2)
ax1.set_xlabel("Year")
ax1.set_ylabel("Pages with Failures (%)")
ax1.set_title("WCAG Failure Rate Over Time")
ax1.set_ylim(90, 100)

ax2.plot(trends["years"], trends["low_contrast_pct"], 'o-', label="Low Contrast", linewidth=2)
ax2.plot(trends["years"], trends["missing_alt_text_pct"], 's-', label="Missing Alt Text", linewidth=2)
ax2.plot(trends["years"], trends["empty_links_pct"], '^-', label="Empty Links", linewidth=2)
ax2.plot(trends["years"], trends["missing_form_labels_pct"], 'D-', label="Missing Labels", linewidth=2)
ax2.set_xlabel("Year")
ax2.set_ylabel("Pages Affected (%)")
ax2.set_title("Top WCAG Failures Over Time")
ax2.legend()
plt.tight_layout()
plt.show()

In [ ]:
# CMS and framework comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
cms = webaim["cms_performance"]
ax1.barh([c["cms"] for c in cms], [c["avg_errors"] for c in cms], color="#2563eb")
ax1.axvline(x=51, color="#dc2626", linestyle="--", label="Average (51)")
ax1.set_xlabel("Average Errors per Page")
ax1.set_title("Accessibility by CMS")
ax1.legend()

fw = webaim["framework_performance"]
ax2.barh([f_item["framework"] for f_item in fw], [f_item["avg_errors"] for f_item in fw], color="#059669")
ax2.axvline(x=51, color="#dc2626", linestyle="--", label="Average (51)")
ax2.set_xlabel("Average Errors per Page")
ax2.set_title("Accessibility by JS Framework")
ax2.legend()
plt.tight_layout()
plt.show()

## 4. Screen Reader User Survey (2024)

WebAIM's 10th survey of screen reader users (1,539 respondents from 7 world regions).

In [ ]:
with open("webaim_screen_reader_survey_2024.json") as f:
    survey = json.load(f)

sr = survey["primary_screen_reader"]
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
labels = [s["name"] for s in sr if s["pct"] >= 2]
sizes = [s["pct"] for s in sr if s["pct"] >= 2]
colors = ["#2563eb", "#059669", "#6366f1", "#d97706", "#dc2626", "#8b5cf6"]
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors[:len(labels)])
ax1.set_title("Primary Screen Reader (Desktop)")

mobile = survey["mobile"]["primary_platform"]
m_labels = [m["name"] for m in mobile if m["pct"] >= 1]
m_sizes = [m["pct"] for m in mobile if m["pct"] >= 1]
ax2.pie(m_sizes, labels=m_labels, autopct='%1.1f%%', colors=["#2563eb", "#059669", "#6366f1"])
ax2.set_title("Primary Mobile Platform")
plt.tight_layout()
plt.show()

In [ ]:
# Most problematic items for screen reader users
problems = survey["problematic_items_ranked"]
fig, ax = plt.subplots(figsize=(12, 6))
ax.barh([p["item"][:40] for p in reversed(problems)], 
        [p["points"] for p in reversed(problems)], color="#dc2626")
ax.set_xlabel("Severity Score (ranking points)")
ax.set_title("Most Problematic Items for Screen Reader Users")
plt.tight_layout()
plt.show()

## 5. ADA Digital Accessibility Lawsuits (2017-2024)

In [ ]:
with open("ada_digital_lawsuits.json") as f:
    lawsuits = json.load(f)

fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(lawsuits["years"], lawsuits["total_lawsuits"], color="#dc2626")
ax.set_xlabel("Year")
ax.set_ylabel("Number of Lawsuits")
ax.set_title("ADA Digital Accessibility Lawsuits (2017-2024)")
for x, y in zip(lawsuits["years"], lawsuits["total_lawsuits"]):
    ax.text(x, y + 50, str(y), ha='center', fontsize=9)
plt.tight_layout()
plt.show()

print(f"2024: {lawsuits['breakdown_2024']['federal_court']:,} federal, "
      f"{lawsuits['breakdown_2024']['state_court']:,} state")
print(f"E-commerce sites targeted: {lawsuits['breakdown_2024']['ecommerce_pct']}%")

## 6. VizWiz: Visual Questions from Blind Users

4,319 images taken by blind users with questions about what they see.

In [ ]:
vizwiz = pd.read_csv("vizwiz_val_annotations.csv")
print(f"Records: {len(vizwiz):,}")
print(f"Answerable: {vizwiz['answerable'].sum():,} ({vizwiz['answerable'].mean()*100:.1f}%)")
print(f"Answer types:\n{vizwiz['answer_type'].value_counts()}")
print(f"\nSample questions:")
for _, row in vizwiz.sample(5, random_state=42).iterrows():
    print(f"  Q: {row['question'][:80]}")
    print(f"  Type: {row['answer_type']}, Answerable: {'Yes' if row['answerable'] else 'No'}\n")

## 7. WLASL: Word-Level American Sign Language

Index of 2,000 ASL signs with video references.

In [ ]:
wlasl = pd.read_csv("wlasl_index.csv")
print(f"Signs: {len(wlasl):,}")
print(f"Columns: {list(wlasl.columns)}")
wlasl.head(10)

---

**Author:** Luke Steuber | [lukesteuber.com](https://lukesteuber.com) | [@lukesteuber.com](https://bsky.app/profile/lukesteuber.com)

**Source:** [Data Trove](https://dr.eamer.dev/datavis/data_trove/) at dr.eamer.dev